# uWell. VLP

Vertical lift perfomance - vertical flow perfomance - кривая лифта - кривая оттока

В данном ноутбуке проводится анализ чувствительности от различных параметров для простой вертикальной скважины

Т.к. расчеты идут долго, для ускорения применяется многопоточность

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature
import uniflocpy.uReservoir.IPR_simple_line as IPR_simple_line
import uniflocpy.uTools.plotly_workflow as plotly_workflow 
import uniflocpy.uValidation.python_api as python_api

In [ ]:
amount_of_threads = 6
from multiprocessing import Pool

In [ ]:
import uniflocpy.uWell.mp_test as mp_test

In [ ]:
well_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 0.7,

"h_intake_mes_m" :1211,
"h_intake_vert_m" : 1211,
"h_bottomhole_mes_m" : 1757, 
"h_bottomhole_vert_m": 1757,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':155.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":240,
"fw_on_surface_perc":25,
            
"step_lenth_in_calc_along_wellbore_m":50,
                "without_annulus_space":False,
            
"fluid":1, 
 "reservoir":0,
"pipe":0,
"temp_corr":1,
"solver_using":1}

In [ ]:
def create_data_to_mp(parameter_name_to_fill, parameter_values_to_fill, 
                      parameter_name_to_iterate="qliq_on_surface_m3day",
                      parameter_name_to_extract='p_bottomhole_bar',
                      well_data=well_data, 
                      amount_of_threads=amount_of_threads, defined_range=range(1, 300, 15)):
    to_mp = []
    for i,j in zip(range(amount_of_threads), parameter_values_to_fill):
        this_well_data = well_data.copy()
        this_well_data[parameter_name_to_fill] = j
        data = [self_flow_well.self_flow_well, this_well_data,defined_range , parameter_name_to_iterate,
                                     parameter_name_to_extract, parameter_name_to_fill, j]
        to_mp.append(data)
    return to_mp

def run_calculation(to_mp, func = mp_test.calc_well_with_one_parameter, amount_of_threads=amount_of_threads):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            result = p.map(func, to_mp)
            
    result_df  = None
    for i in result:
        try:
            result_df = result_df.join(i, how = 'outer')
        except:
            result_df = i.copy()
    return result_df

In [ ]:
to_mp = create_data_to_mp("fw_on_surface_perc", [0, 10, 25, 50, 70, 100],defined_range = range(1,300,50)) 

In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)


In [ ]:
to_mp = create_data_to_mp("rsb_m3m3", [20, 30, 50,  100, 200,400],defined_range = range(1,300,25)) 

In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)


In [ ]:
to_mp = create_data_to_mp("d_tube_inner_m", [0.04, 0.05, 0.06,  0.07, 0.08,0.09],defined_range = range(1,300,15)) 

In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)


In [ ]:
to_mp = create_data_to_mp("step_lenth_in_calc_along_wellbore_m", 
                          [10, 20, 40,  80, 150,300],defined_range = range(1,300,15)) 


In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)


In [ ]:
to_mp = create_data_to_mp("t_bottomhole_c", 
                          [20,  40, 60, 80, 100,120],defined_range = range(1,300,15)) 


In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)


In [ ]:
to_mp = create_data_to_mp("gamma_oil", 
                          [0.65,  0.7, 0.8,0.85, 0.9, 0.95],defined_range = range(1,300,20)) 


In [ ]:
result_df = run_calculation(to_mp)

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Анализ чувствительности', 'check.html', iplot_option = True)
